In [1]:
import pygmo as pg
import pygmo_plugins_nonfree as pg7
import PyKEP as pk
import numpy as np
from PyKEP.trajopt import *
%matplotlib

/usr/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: to-Python converter for pagmo::population already registered; second conversion method ignored.
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: to-Python converter for pagmo::archipelago already registered; second conversion method ignored.
  return f(*args, **kwds)


Using matplotlib backend: Qt5Agg


In [2]:
uda = pg7.snopt7(True, "/usr/lib/libsnopt7_c.so")
uda.set_integer_option("Major iterations limit", 4000)
uda.set_integer_option("Iterations limit", 40000)
uda.set_numeric_option("Major optimality tolerance", 1e-2)
uda.set_numeric_option("Major feasibility tolerance", 1e-8)
algo = pg.algorithm(uda)

In [3]:
# base
mass = 1000
thrust = 0.3
isp = 2500
mu = pk.MU_SUN
freemass = True
freetime = True
alpha = 1
bound = True
atol = 1e-10
rtol = 1e-10
t0lb = 0
t0ub = 0.01
Tlb = 100
Tub = 2000

# or2or
t0 = pk.epoch(0)
tf = pk.epoch(1000)
p0 = pk.planet.jpl_lp("earth")
pf = pk.planet.jpl_lp("mars")
r0, v0 = p0.eph(t0)
rf, vf = pf.eph(tf)
elem0=[
    149598261129.93335,
    0.016711230601231957,
    2.640492490927786e-07,
    3.141592653589793,
    4.938194050401601,
    0
]
elemf=[
    227943822376.03537,
    0.09339409892101332,
    0.032283207367640024,
    0.8649771996521327,
    5.000312830124232,
    0
]
x0 = np.hstack((r0, v0, [mass]))
xf = np.hstack((rf, vf, [mass/10]))

# Indirect Point to Point

In [ ]:
udp = pk.trajopt.indirect_pt2pt(
    x0, xf, mass, thrust, isp, mu, t0lb, t0ub, Tlb, Tub, atol=1e-8, rtol=1e-8
)
prob = pg.problem(udp)

In [ ]:
pop = pg.population(prob, 1)
pop = algo.evolve(pop)

In [ ]:
udp.plot_traj(pop.champion_x)

In [ ]:
udp.plot_control(pop.champion_x)
pop.champion_x

# Indirect Planet to Planet

In [ ]:
udp = pk.trajopt.indirect_pl2pl("earth", "mars", mass, thrust, isp, 1e-8, 1e-8, t0lb, t0ub, Tlb, Tub, freemass=True, freetime=True, alpha=1, bound=True)
prob = pg.problem(udp)

In [ ]:
pop = pg.population(udp, 1)
pop = algo.evolve(pop)

In [ ]:
udp.plot_traj(pop.champion_x)
udp.plot_control(pop.champion_x)

# Indirect Orbit to Orbit

In [4]:
Mlim = 2*3.15
udp = pk.trajopt.indirect_or2or(elem0, elemf, mass, thrust, isp, 1e-8, 1e-8, Tlb, Tub, -Mlim, Mlim, -Mlim, Mlim)
prob = pg.problem(udp)

True True 1 True


In [ ]:
pop = pg.population(prob, 1)
pop = algo.evolve(pop)

In [ ]:
udp.plot_traj(pop.champion_x)

In [ ]:
udp.plot_control(pop.champion_x)

# Indirect Point to Orbit

In [10]:
udp = pk.trajopt.indirect_pt2or(
    x0, elemf, mass, thrust, isp, 1e-8, 1e-8, Tlb, Tub, -Mlim, Mlim)
prob = pg.problem(udp)

True True 1 True


In [11]:
pop = pg.population(prob, 1)
pop = algo.evolve(pop)

/usr/lib/python3.6/site-packages/scipy/integrate/_ode.py:1035: UserWarning: dopri5: step size becomes too small
  self.messages.get(idid, 'Unexpected idid=%s' % idid))


RuntimeError: couldn't create a NumPy array: the 'PyArray_SimpleNew()' function failed

In [12]:
udp.plot_traj(pop.champion_x)

/usr/lib/python3.6/site-packages/scipy/integrate/_ode.py:1035: UserWarning: dopri5: step size becomes too small
  self.messages.get(idid, 'Unexpected idid=%s' % idid))
/usr/lib/python3.6/site-packages/scipy/integrate/_ode.py:1035: UserWarning: dopri5: larger nmax is needed
  self.messages.get(idid, 'Unexpected idid=%s' % idid))


In [8]:
udp.plot_control(pop.champion_x)

/usr/lib/python3.6/site-packages/scipy/integrate/_ode.py:1035: UserWarning: dopri5: larger nmax is needed
  self.messages.get(idid, 'Unexpected idid=%s' % idid))
